In [1]:
# ----------------------------------------------------------
# Step 0 – Imports and Display Setup
# ----------------------------------------------------------

import os
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx


In [3]:
# ----------------------------------------------------------
# Step 1 – Define File Paths and Load Processed Inputs
# ----------------------------------------------------------


# Define base directory and processed/raw paths
base_dir = "/Users/rosstaylor/Downloads/Research Project/Code Folder/diagnostic-modality-demand/diagnostic-modality-demand"
data_dir = os.path.join(base_dir, "data")
processed_dir = os.path.join(data_dir, "processed")
raw_dir = os.path.join(data_dir, "raw")

# Load LSOA with 2SFCA baseline scores
gdf_lsoa = gpd.read_file(os.path.join(processed_dir, "LSOA_CT_2SFCA_Accessibility_2024.gpkg"))

# Load scanner site supply GeoDataFrame
gdf_sites = gpd.read_file(os.path.join(processed_dir, "CT_Scanner_Sites_With_SupplyRatios.gpkg"))

# Load travel matrix (used unchanged from raw)
df_travel = pd.read_csv(os.path.join(raw_dir, "LSOA_to_LSOA_complete.csv"))

print("Baseline data loaded. Ready to define scenario.")


Baseline data loaded. Ready to define scenario.
